In [1]:
from camera_roboter import CameraRoboter
from math import pi
import rospy

In [2]:
CameraRoboterI = CameraRoboter(load_camera=True)

[ INFO] [1685436699.035521600]: Loading robot model 'panda'...
[ INFO] [1685436700.113140605]: Ready to take commands for planning group panda_arm.
initCamera returns 213
SetColorMode IS_CM_BGR8_PACKED returns 1
setExposure returns 1
AOI IS_AOI_IMAGE_SET_AOI returns 1
AllocImageMem returns 1
SetImageMem returns 1
CaptureVideo returns 1
============ Planning frame: panda_link0
============ End effector link: panda_link8
============ Available Planning Groups: ['panda_arm']
============ Printing robot state
joint_state: 
  header: 
    seq: 0
    stamp: 
      secs: 0
      nsecs:         0
    frame_id: "panda_link0"
  name: 
    - panda_joint1
    - panda_joint2
    - panda_joint3
    - panda_joint4
    - panda_joint5
    - panda_joint6
    - panda_joint7
  position: [-6.412450848208544e-05, -0.7857621713021538, 9.957918090577491e-05, -2.3556000207512082, -0.00019125962115885308, 1.571155472954114, -0.7854259103635947]
  velocity: []
  effort: []
multi_dof_joint_state: 
  header: 
    

In [3]:
CameraRoboterI.calibration()
CameraRoboterI.return_to_ready_pose()

In [4]:
dict_info = dict()
dict_info['id'] = "5_008"
dict_info['speed'] = "500"
dict_info['rotate_speed'] = "2900"
dict_info['date'] = "06.11.19"

training_label = list(range(6))
training_label[0] = 0
training_label[1] = 0
training_label[2] = 0
training_label[3] = 0
training_label[4] = 0
training_label[5] = 0

dict_info['training_label'] = training_label
dict_info

{'id': '5_008',
 'speed': '500',
 'rotate_speed': '2900',
 'date': '06.11.19',
 'training_label': [0, 0, 0, 0, 0, 0]}

In [5]:
CameraRoboterI.DataManager.set_basic_info(dict_info)

In [6]:
# Helligkeit

CameraRoboterI.DataManager.set_brightness(50)

CameraRoboterI.move_pose(CameraRoboterI.x_init, CameraRoboterI.y_init, CameraRoboterI.z_init, CameraRoboterI.roll_angle_init, CameraRoboterI.pitch_angle_init, CameraRoboterI.yaw_angel_init, save_image=True)
CameraRoboterI.move_linear_trajectory(0.016, 5, save_image=True)

CameraRoboterI.return_to_ready_pose()

image is saved als name: images/5_008_500_2900/0.42_50_0_0_0_0_0_0.jpg
image is saved als name: images/5_008_500_2900/0.44_50_0_0_0_0_0_0.jpg
image is saved als name: images/5_008_500_2900/0.45_50_0_0_0_0_0_0.jpg
image is saved als name: images/5_008_500_2900/0.47_50_0_0_0_0_0_0.jpg
image is saved als name: images/5_008_500_2900/0.48_50_0_0_0_0_0_0.jpg
image is saved als name: images/5_008_500_2900/0.5_50_0_0_0_0_0_0.jpg


In [12]:
# Drehen um x

CameraRoboterI.DataManager.set_brightness(50)

CameraRoboterI.move_pose(CameraRoboterI.x_init, CameraRoboterI.y_init, CameraRoboterI.z_init, CameraRoboterI.roll_angle_init, CameraRoboterI.pitch_angle_init, CameraRoboterI.yaw_angel_init)

angle = CameraRoboterI.DataManager.set_rotate_angle_x(pi/75)
CameraRoboterI.change_angle(angle)
CameraRoboterI.get_image(take_image=True, image_index=0)
CameraRoboterI.move_linear_trajectory(0.016, 5, save_image=True)

CameraRoboterI.return_to_ready_pose()

image is saved als name: images/5_008_500_2900/0.42_50_0.04_0_0_0_0_0.jpg
image is saved als name: images/5_008_500_2900/0.44_50_0.04_0_0_0_0_0.jpg
image is saved als name: images/5_008_500_2900/0.45_50_0.04_0_0_0_0_0.jpg
image is saved als name: images/5_008_500_2900/0.47_50_0.04_0_0_0_0_0.jpg
image is saved als name: images/5_008_500_2900/0.48_50_0.04_0_0_0_0_0.jpg
image is saved als name: images/5_008_500_2900/0.5_50_0.04_0_0_0_0_0.jpg


In [19]:
# Drehen um z

CameraRoboterI.DataManager.set_brightness(50)

angle = CameraRoboterI.DataManager.set_rotate_angle_z(-pi/25)
CameraRoboterI.rotate(angle)
CameraRoboterI.move_position(CameraRoboterI.x_init, CameraRoboterI.y_init, CameraRoboterI.z_init, save_image=True)
CameraRoboterI.move_linear_trajectory(0.016, 5, save_image=True)

CameraRoboterI.return_to_ready_pose()

image is saved als name: images/5_008_500_2900/0.42_50_0.04_-0.13_0_0_0_0.jpg
image is saved als name: images/5_008_500_2900/0.44_50_0.04_-0.13_0_0_0_0.jpg
image is saved als name: images/5_008_500_2900/0.45_50_0.04_-0.13_0_0_0_0.jpg
image is saved als name: images/5_008_500_2900/0.47_50_0.04_-0.13_0_0_0_0.jpg
image is saved als name: images/5_008_500_2900/0.48_50_0.04_-0.13_0_0_0_0.jpg
image is saved als name: images/5_008_500_2900/0.5_50_0.04_-0.13_0_0_0_0.jpg


In [6]:
# displancement in z, manuell blur

CameraRoboterI.DataManager.set_brightness(50)

CameraRoboterI.move_position(CameraRoboterI.x_init, CameraRoboterI.y_init, CameraRoboterI.z_init)
CameraRoboterI.get_video_stream()

CameraRoboterI.move_pose(CameraRoboterI.x_init, CameraRoboterI.y_init, CameraRoboterI.z_init, CameraRoboterI.roll_angle_init, CameraRoboterI.pitch_angle_init, CameraRoboterI.yaw_angel_init, save_image=True)
CameraRoboterI.move_linear_trajectory(0.016, 5, save_image=True)

CameraRoboterI.return_to_ready_pose()

StopLiveVideo returns 0
ExitCamera returns 0
image is saved als name: images/5_008_500_2900/0.43_50_0_0_0_0_0_0.jpg
image is saved als name: images/5_008_500_2900/0.45_50_0_0_0_0_0_0.jpg
image is saved als name: images/5_008_500_2900/0.46_50_0_0_0_0_0_0.jpg
image is saved als name: images/5_008_500_2900/0.48_50_0_0_0_0_0_0.jpg
image is saved als name: images/5_008_500_2900/0.49_50_0_0_0_0_0_0.jpg
image is saved als name: images/5_008_500_2900/0.51_50_0_0_0_0_0_0.jpg


In [3]:
CameraRoboterI.close_camera()

StopLiveVideo returns 1
ExitCamera returns 1
